# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

* The dataset random and provides a sample of approximately 500 cities of interest and doesn’t fully represent all of the worldwide cities in the northern and southern hemisphere.  Of the data retrieved however, ideal cities meeting our set criteria for ideal conditions occurred in the northern hemisphere with six events, whereas only four appeared in the southern hemisphere.

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import cartopy.crs as ccrs
import geoviews as gv
import requests
#from citipy import citipy


# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,navashino,55.5292,42.2003,16.09,98,67,3.21,RU,1685045620
1,1,yellowknife,62.4560,-114.3525,14.20,54,44,2.68,CA,1685045362
2,2,khandyga,62.6667,135.6000,4.61,85,99,0.19,RU,1685045620
3,3,grytviken,-54.2811,-36.5092,0.73,98,97,2.42,GS,1685045620
4,4,invercargill,-46.4000,168.3500,13.47,70,99,5.37,NZ,1685045620


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display
#url = "https://api.geoapify.com/v1/geocode/search?REQUEST_PARAMS"

# Plot size based on the humidity for each latitude/longitude point for every city in the city_data_df. 
map_plot_1 = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    c = "City",
    geo = True,
    tiles = "OSM",
    colormap = "hsv",
    title = "City Map",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5   
)

# Display the map
map_plot_1

#hvplot.save(map_plot_1, 'Map1.html')

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values for the max temp, wind speed, and cloudiness.
combined_data = (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] >21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)

# Create a new data frame for the ideal cities.
ideal_cities_df = city_data_df[combined_data]
            
# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,westport,41.1415,-73.3579,21.11,41,0,3.60,US,1685045631
188,188,arraial do cabo,-22.9661,-42.0278,23.92,83,0,4.12,BR,1685045636
219,219,regente feijo,-22.2214,-51.3028,22.80,64,0,3.10,BR,1685045639
262,262,maintirano,-18.0667,44.0167,21.47,69,0,3.22,MG,1685045643
335,335,mashhad,36.2970,59.6062,23.08,38,0,2.06,IR,1685045655
363,363,san juan,-31.5375,-68.5364,23.61,37,0,1.93,AR,1685045658
399,399,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1685045661
407,407,zahedan,29.4963,60.8629,24.95,19,0,2.06,IR,1685045662
529,529,senj,44.9894,14.9058,22.37,85,0,3.88,HR,1685045673
546,546,lindsay,36.2030,-119.0882,25.43,51,0,2.57,US,1685045675


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_cities_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
133,westport,US,41.1415,-73.3579,41,
188,arraial do cabo,BR,-22.9661,-42.0278,83,
219,regente feijo,BR,-22.2214,-51.3028,64,
262,maintirano,MG,-18.0667,44.0167,69,
335,mashhad,IR,36.2970,59.6062,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(name_address)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
westport - nearest hotel: Norwalk Inn and Conference Center
arraial do cabo - nearest hotel: No hotel found
regente feijo - nearest hotel: No hotel found
maintirano - nearest hotel: Hotel Safari
mashhad - nearest hotel: هتل تعاون
san juan - nearest hotel: Hotel San Francisco
remire-montjoly - nearest hotel: Complexe Belova
zahedan - nearest hotel: هتل امین
senj - nearest hotel: Laura apartmanban
lindsay - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
133,westport,US,41.1415,-73.3579,41,Norwalk Inn and Conference Center
188,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
219,regente feijo,BR,-22.2214,-51.3028,64,No hotel found
262,maintirano,MG,-18.0667,44.0167,69,Hotel Safari
335,mashhad,IR,36.2970,59.6062,38,هتل تعاون
363,san juan,AR,-31.5375,-68.5364,37,Hotel San Francisco
399,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
407,zahedan,IR,29.4963,60.8629,19,هتل امین
529,senj,HR,44.9894,14.9058,85,Laura apartmanban
546,lindsay,US,36.2030,-119.0882,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Plot the latitude/longitude points for every country in the hotel dataframe.
map_plot_2 = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    c = "Hotel Name",
    geo = True,
    hover_cols="Country",
    tiles = "OSM",
    colormap = "hsv",
    title = "City Map",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    alpha = 0.5   
)

# Display the map
map_plot_2
#hvplot.save(map_plot_2, 'Map2.html')


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)